# Efficient Anomaly Detection in Industrial Images using Transformers with Dynamic Tanh

## Imports

In [ ]:
from typing import Literal

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
import torch
from torch.utils.data import DataLoader

from modules import data, globals, vtae

## Globals

In [ ]:
# Dataset
dataset: Literal['MVTech', 'BTAD'] = 'MVTech'
product: data.ProductType = 'bottle'
resize_dim: int = 550
crop_dim: int = 512

# Model
patch_side: int = 64
latent_channels: int = 8
heads: int = 1
depth: int = 1
caps_per_patch: int = 1
caps_dim: int = 1
caps_iterations: int = 1
ff_dim: int = 512
mdn_components: int = 50
noise: float = 0.2
loss_weights: tuple[float, float, float] = (5., 0.5, 1.)
lr: float = 1e-4
weight_decay: float = 1e-4
use_dytanh: bool = False

# Training
epochs: int = 400
batch_size: int = 8

In [ ]:
# Run name
run_name: str = 'vtae_dytanh' if use_dytanh else 'vtae_ln'

In [ ]:
pl.seed_everything(42, workers = True)

## Data

In [ ]:
train_loader: DataLoader[tuple[torch.Tensor]]
val_loader: DataLoader[tuple[torch.Tensor]]
test_loader: DataLoader[tuple[torch.Tensor, torch.Tensor]]

train_loader, val_loader, test_loader = data.get_loaders(dataset,
                                                         product,
                                                         crop_dim = (crop_dim, crop_dim),
                                                         resize_dim = (resize_dim, resize_dim),
                                                         batch_size = batch_size
                                                         )

## Network

In [ ]:
model: vtae.VTAE = vtae.VTAE(image_shape = (3, crop_dim, crop_dim),
                             patch_shape = (patch_side, patch_side),
                             latent_channels = latent_channels,
                             heads = heads,
                             depth = depth,
                             caps_per_patch = caps_per_patch,
                             caps_dim = caps_dim,
                             caps_iterations = caps_iterations,
                             ff_dim = ff_dim,
                             mdn_components = mdn_components,
                             noise = noise,
                             loss_weights = loss_weights,
                             lr = lr,
                             weight_decay = weight_decay,
                             use_dytanh = use_dytanh
                             )

## Train

In [ ]:
# Logger
logger: TensorBoardLogger = TensorBoardLogger(globals.LOG_DIR / dataset / product, name = run_name)

# Train the model
trainer: pl.Trainer = pl.Trainer(max_epochs = epochs,
                                 precision = 'bf16-mixed',
                                 log_every_n_steps = len(train_loader),
                                 logger = logger,
                                 enable_checkpointing = False   # In order to not alter the times, we save only at the end
                                 )
trainer.fit(model, train_loader, val_loader)

In [ ]:
# Save the model
trainer.save_checkpoint(globals.CHECKPOINT_DIR / dataset / product / f'{run_name}.ckpt')

## Evaluation

In [ ]:
trainer.test(model, test_loader)